In [1]:
import numpy as np
import pandas as pd
#for reading in data properly
import ast
import json

import gensim
from gensim.models import Doc2Vec
from gensim.models import Word2Vec
from gensim.models.doc2vec import TaggedDocument

from sklearn.model_selection import train_test_split
from sklearn import utils

import re

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

stop_words = set(stopwords.words('english'))

In [2]:
all_data = pd.read_csv('train.csv')
all_data = all_data.dropna(subset=['overview', 'genres']) #drop cols without overview or genre (data we use or labels)
genre_set = {'Comedy'}

In [3]:
def text_to_list(x):
    if pd.isna(x):
        return ''
    else:
        return ast.literal_eval(x)

def parse_json(x):
    try:
        return json.loads(x.replace("'", '"'))[0]['name']
    except:
        return ''
    
def parse_all_genres_json(x):
    try:
        json_genres = json.loads(x.replace("'", '"'))
        numElems = len(json_genres)
        for i in range(numElems):
            genre_set.add(json_genres[i]['name'])
    except:
        return ''
    
def parse_genres_json(x):
    try:
        json_genres = json.loads(x.replace("'", '"'))
        numElems = len(json_genres)
        ret = [0]*len(genre_dict) #20 0s
        for i in range(numElems):
            ret[genre_dict[(json_genres[i]['name'])]] = 1
        return ret
    except:
        return ''
    

def get_labels_as_strs(x):
    try:
        json_genres = json.loads(x.replace("'", '"'))
        numElems = len(json_genres)
        ret = []#20 0s
        for i in range(numElems):
            ret.append(json_genres[i]['name'])
        return ret
    except:
        return ''

In [4]:
 def getAllGenres():
    full_data = pd.read_csv('train.csv')

    y = full_data['genres']
    y.apply(parse_all_genres_json)

In [5]:
getAllGenres()#populate the genre set

In [6]:
#get set to dictionary for indexing of target vectors
genre_dict = {}
index = 0
for genre in genre_set:
    genre_dict[genre] = index
    index += 1

In [7]:
def getGenresVects():
    y = all_data['genres']
    ret = y.apply(parse_genres_json)
    all_data['genres_vect'] = ret
    label_strs = y.apply(get_labels_as_strs)
    all_data['genres_labels'] = label_strs
    return ret

In [8]:
labels_vects = getGenresVects() #get label vectors for genres indexed by indexes in genre_dict

In [9]:
#put to lower case, remove punctation
def cleanText(text):
    text = re.sub(r'[^a-z A-Z0-9]', "", text) #maybe shouldn't remove punction between words here?
    text = text.lower()
    return text
all_data['cleanOverview'] = all_data['overview'].apply(cleanText)

In [10]:
#logistic regression data
lr_data = all_data[['cleanOverview', 'genres_labels', 'genres_vect', 'overview']]

In [11]:
train, test = train_test_split(lr_data, test_size=0.2, random_state=42)

CNN STUFF here

In [12]:
#get word embeddings
x = train['cleanOverview'].values.tolist()
y = train['genres_vect']

In [13]:
x_test = test['cleanOverview'].values.tolist()
y_test = test['genres_vect']

In [14]:
y_train = y.tolist()
y_train = np.array(y_train)

In [15]:
y_test = y_test.tolist()
y_test = np.array(y_test)

In [16]:
tok = [word_tokenize(sent) for sent in x]

In [17]:
model = Word2Vec(tok, min_count = 1, size=32)

In [18]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer()
tokenizer.fit_on_texts(x)
sequences = tokenizer.texts_to_sequences(x)

x_train_seq = pad_sequences(sequences, maxlen=200)

Using TensorFlow backend.


In [19]:
test_sequences = tokenizer.texts_to_sequences(x_test)
x_test_seq = pad_sequences(test_sequences, maxlen=200)

In [20]:
embeddings_index = {}
for w in model.wv.vocab.keys():
    embeddings_index[w] = model.wv[w]


num_words = 100000
embedding_matrix = np.zeros((num_words, 32))
for word, i in tokenizer.word_index.items():
    if i >= num_words:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [21]:
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Flatten
from keras.layers.embeddings import Embedding

model_cnn_01 = Sequential()
e = Embedding(100000, 32, weights=[embedding_matrix], input_length=200, trainable=False)
model_cnn_01.add(e)
model_cnn_01.add(Conv1D(filters=100, kernel_size=2, padding='valid', activation='relu', strides=1))
model_cnn_01.add(GlobalMaxPooling1D())
model_cnn_01.add(Dense(256, activation='relu'))
model_cnn_01.add(Dense(20, activation='sigmoid'))
model_cnn_01.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#model_cnn_01.fit(x_train_seq, y_train, validation_data=(x_val_seq, y_validation), epochs=5, batch_size=32, verbose=2)
model_cnn_01.fit(x_train_seq, y_train, epochs=500, batch_size=32, verbose=2)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Epoch 1/500
 - 2s - loss: 0.3388 - acc: 0.8614
Epoch 2/500
 - 1s - loss: 0.3156 - acc: 0.8769
Epoch 3/500
 - 1s - loss: 0.3163 - acc: 0.8747
Epoch 4/500
 - 1s - loss: 0.3144 - acc: 0.8759
Epoch 5/500
 - 1s - loss: 0.3144 - acc: 0.8761
Epoch 6/500
 - 1s - loss: 0.3148 - acc: 0.8745
Epoch 7/500
 - 1s - loss: 0.3143 - acc: 0.8749
Epoch 8/500
 - 1s - loss: 0.3142 - acc: 0.8757
Epoch 9/500
 - 2s - loss: 0.3135 - acc: 0.8752
Epoch 10/500
 - 1s - loss: 0.3138 - acc: 0.8764
Epoch 11/500
 - 1s - loss: 0.3142 - acc: 0.8754
Epoch 12/500
 - 1s - loss: 0.3139 - acc: 0.8747
Epoch 13/500
 - 1s - loss: 0.3135 - acc: 0.8752
Epoch 14/500
 - 1s - loss: 0.3135 - acc: 0.8761
Epoch 15/500
 - 1s - loss: 0.3131 - acc: 0.8754
Epoch 16/500
 - 1s - loss: 0.3132 - acc: 0.8756
Epoch 17/500
 - 1s - loss: 0.3134 - acc: 0.

 - 1s - loss: 0.2932 - acc: 0.8805
Epoch 154/500
 - 1s - loss: 0.2921 - acc: 0.8805
Epoch 155/500
 - 1s - loss: 0.2922 - acc: 0.8814
Epoch 156/500
 - 1s - loss: 0.2920 - acc: 0.8812
Epoch 157/500
 - 1s - loss: 0.2917 - acc: 0.8815
Epoch 158/500
 - 1s - loss: 0.2920 - acc: 0.8808
Epoch 159/500
 - 1s - loss: 0.2902 - acc: 0.8813
Epoch 160/500
 - 2s - loss: 0.2910 - acc: 0.8817
Epoch 161/500
 - 1s - loss: 0.2913 - acc: 0.8810
Epoch 162/500
 - 1s - loss: 0.2898 - acc: 0.8814
Epoch 163/500
 - 1s - loss: 0.2897 - acc: 0.8812
Epoch 164/500
 - 1s - loss: 0.2888 - acc: 0.8818
Epoch 165/500
 - 1s - loss: 0.2883 - acc: 0.8819
Epoch 166/500
 - 1s - loss: 0.2881 - acc: 0.8819
Epoch 167/500
 - 1s - loss: 0.2891 - acc: 0.8823
Epoch 168/500
 - 2s - loss: 0.2869 - acc: 0.8822
Epoch 169/500
 - 1s - loss: 0.2867 - acc: 0.8827
Epoch 170/500
 - 1s - loss: 0.2875 - acc: 0.8820
Epoch 171/500
 - 1s - loss: 0.2868 - acc: 0.8823
Epoch 172/500
 - 1s - loss: 0.2866 - acc: 0.8828
Epoch 173/500
 - 1s - loss: 0.2858

Epoch 321/500
 - 1s - loss: 0.2419 - acc: 0.8994
Epoch 322/500
 - 1s - loss: 0.2423 - acc: 0.8995
Epoch 323/500
 - 1s - loss: 0.2425 - acc: 0.8986
Epoch 324/500
 - 1s - loss: 0.2406 - acc: 0.8990
Epoch 325/500
 - 1s - loss: 0.2405 - acc: 0.8999
Epoch 326/500
 - 1s - loss: 0.2414 - acc: 0.8988
Epoch 327/500
 - 1s - loss: 0.2403 - acc: 0.8996
Epoch 328/500
 - 2s - loss: 0.2406 - acc: 0.8992
Epoch 329/500
 - 1s - loss: 0.2394 - acc: 0.9006
Epoch 330/500
 - 1s - loss: 0.2386 - acc: 0.9013
Epoch 331/500
 - 1s - loss: 0.2393 - acc: 0.8999
Epoch 332/500
 - 1s - loss: 0.2388 - acc: 0.9006
Epoch 333/500
 - 1s - loss: 0.2387 - acc: 0.9010
Epoch 334/500
 - 2s - loss: 0.2374 - acc: 0.9017
Epoch 335/500
 - 1s - loss: 0.2379 - acc: 0.9014
Epoch 336/500
 - 2s - loss: 0.2403 - acc: 0.9000
Epoch 337/500
 - 1s - loss: 0.2373 - acc: 0.9015
Epoch 338/500
 - 1s - loss: 0.2380 - acc: 0.9011
Epoch 339/500
 - 1s - loss: 0.2366 - acc: 0.9014
Epoch 340/500
 - 1s - loss: 0.2373 - acc: 0.9013
Epoch 341/500
 - 1s 

 - 1s - loss: 0.2042 - acc: 0.9158
Epoch 489/500
 - 1s - loss: 0.2045 - acc: 0.9157
Epoch 490/500
 - 1s - loss: 0.2034 - acc: 0.9173
Epoch 491/500
 - 1s - loss: 0.2034 - acc: 0.9164
Epoch 492/500
 - 1s - loss: 0.2042 - acc: 0.9154
Epoch 493/500
 - 2s - loss: 0.2049 - acc: 0.9146
Epoch 494/500
 - 2s - loss: 0.2026 - acc: 0.9163
Epoch 495/500
 - 2s - loss: 0.2018 - acc: 0.9162
Epoch 496/500
 - 1s - loss: 0.2026 - acc: 0.9168
Epoch 497/500
 - 1s - loss: 0.2015 - acc: 0.9172
Epoch 498/500
 - 1s - loss: 0.2016 - acc: 0.9180
Epoch 499/500
 - 1s - loss: 0.2000 - acc: 0.9180
Epoch 500/500
 - 1s - loss: 0.2023 - acc: 0.9171


In [22]:
res = model_cnn_01.predict(x_test_seq)

In [23]:
label_predictions = []
for i in range(res.shape[0]):
    pred = [0]*20
    for j in range(res.shape[1]):
        if res[i][j] >= .5:
            pred[j] = 1
    label_predictions.append(pred)

In [24]:
y_test[:,1].sum()

319

In [25]:
label_predictions = np.array(label_predictions)
label_predictions[:,1].sum()

286

In [48]:
#intersection of predicted and actual labels divided by uniont for each datapoint tested on
#sum those and then divide by number of datapoints
#vectorized for speed
def multi_label_accuracy(real_labels_matrix, predictions_labels_matrix):
    #binary so set intersection is and operator
    intersection = real_labels_matrix & predictions_labels_matrix
    #set union for binary is same as or operator
    union = real_labels_matrix | predictions_labels_matrix
    #sum(array.T) gets number of 1s in row
    row_wise_accuracy = sum(intersection.T) / sum(union.T)
    return sum(row_wise_accuracy) / real_labels_matrix.shape[0]

In [49]:
multi_label_accuracy(y_test, label_predictions)

0.1580501714923826